In [1]:
from pathlib import Path
from kinfraglib import utils, filters
import docking_utils
from rdkit import Chem
from pathlib import Path
from rdkit import Chem
import utils_eval

import json

with open('definitions.json', 'rt') as json_file:
    definitions = json.load(json_file)

# Definitions
HERE = Path().resolve()
PATH_DATA = Path('../KinFragLib/data')
#PATH_TO_DOCKING_CONFIGS = Path(definitions['DockingConfig']) / definitions['pdbCode']
PATH_TO_SDF_FRAGMENTS = HERE / 'data/fragments' / definitions['pdbCode']
PATH_TO_DOCKING_RESULTS = HERE / 'data/docking' / definitions['pdbCode']
PATH_TO_HYDE_RESULTS = HERE / 'data/scoring' / definitions['pdbCode']
PATH_TO_TEMPLATES =  HERE / 'data/templates' / definitions['pdbCode']


# Access fragment library
fragment_library_original = utils.read_fragment_library(PATH_DATA / "fragment_library")
fragment_library = filters.prefilters.pre_filters(fragment_library_original)



## Show that AP fragment of 3amb was filtered in QED step

In [2]:
filters_ = [docking_utils.Filter(name, values) for name, values in definitions['Filters'].items()]
print(fragment_library['AP'][fragment_library['AP'].smiles == 'Cc1cc(N)[nH]n1'].index.size)
for filter in filters_:
    l_before = [sp + ': ' + str(len(fragment_library[sp])) for sp in fragment_library.keys()]
    print(filter.name)
    fragment_library = filter.apply_filter(fragment_library)
    print(fragment_library['AP'][fragment_library['AP'].smiles == 'Cc1cc(N)[nH]n1'].index.size)

1
pains
1
brenk
Number of unwanted substructures in Brenk et al. collection: 104
1
ro3
1
qed
0
syba
0


# Show that 5l4q ligand was dropped

In [3]:
# load 5l4q compounds
data = utils_eval.read_mols('../cluster_results/5l4q120923/5l4q/results.sdf')

In [4]:
data['inchi'] = data.apply(lambda x: Chem.MolToInchi(utils.standardize_mol(x.ROMol)), axis=1)

In [5]:
data_two_frag = data[data.num_fragments ==2].copy().sort_values(['BIOSOLVEIT.HYDE_ESTIMATED_AFFINITY_LOWER_BOUNDARY [nM]']).reset_index(drop=True)

In [6]:
data_two_frag

,ROMol,BIOSOLVEIT.HYDE_ESTIMATED_AFFINITY_LOWER_BOUNDARY [nM],BIOSOLVEIT.HYDE_ESTIMATED_AFFINITY_UPPER_BOUNDARY [nM],BIOSOLVEIT.LOGP,BIOSOLVEIT.MOLECULAR_WEIGHT,BIOSOLVEIT.TPSA,fragment_ids,smiles_ligand,smiles_fragments_dummy,smiles_fragments,BIOSOLVEIT.DOCKING_SCORE,filtered,num_fragments,inchi
0,"<img data-content=""rdkit/molecule"" src=""data:i...",1.318690e-01,1.310200e+01,0.61940,324.406,75.11,"{'AP': 119, 'SE': 219}",CNC(=O)C[NH+]1CCC(c2cnc3[nH]c4cnccc4c3c2)CC1,"{'AP': '[1*]c1cc2c(cn1)[nH]c1ncc([21*])cc12', ...","{'AP': 'c1cnc2[nH]c3cnccc3c2c1', 'SE': 'CNC(=O...",-29.47,0,2,InChI=1S/C18H21N5O/c1-19-17(24)11-23-6-3-12(4-...
1,"<img data-content=""rdkit/molecule"" src=""data:i...",2.100520e-01,2.086990e+01,2.74282,340.404,94.92,"{'AP': 119, 'SE': 9}",Cc1nc(C(C)(O)CO)sc1-c1cnc2[nH]c3cnccc3c2c1,"{'AP': '[1*]c1cc2c(cn1)[nH]c1ncc([21*])cc12', ...","{'AP': 'c1cnc2[nH]c3cnccc3c2c1', 'SE': 'Cc1csc...",-24.89,0,2,"InChI=1S/C17H16N4O2S/c1-9-14(24-16(20-9)17(2,2..."
2,"<img data-content=""rdkit/molecule"" src=""data:i...",2.360840e-01,2.345631e+01,4.80080,384.437,64.96,"{'AP': 119, 'SE': 114}",COc1ccc(OC)c(-c2cc(-c3cnc4[nH]c5cnccc5c4c3)n(C...,"{'AP': '[1*]c1cc2c(cn1)[nH]c1ncc([21*])cc12', ...","{'AP': 'c1cnc2[nH]c3cnccc3c2c1', 'SE': 'COc1cc...",-24.29,0,2,InChI=1S/C23H20N4O2/c1-27-13-15(18-10-16(28-2)...
3,"<img data-content=""rdkit/molecule"" src=""data:i...",2.486100e-01,2.470089e+01,1.97450,374.470,74.17,"{'AP': 119, 'SE': 50}",CC[NH+]1CCN(c2ccc(Nc3cnc4[nH]c5cnccc5c4c3)nc2)CC1,"{'AP': '[1*]c1cc2c(cn1)[nH]c1ncc([21*])cc12', ...","{'AP': 'c1cnc2[nH]c3cnccc3c2c1', 'SE': 'CC[NH+...",-27.23,0,2,InChI=1S/C21H23N7/c1-2-27-7-9-28(10-8-27)16-3-...
4,"<img data-content=""rdkit/molecule"" src=""data:i...",2.687390e-01,2.670076e+01,0.55980,428.537,93.80,"{'AP': 124, 'SE': 194}",C[NH+]1CC[NH+](Cc2ccc3c(-c4cnc(N)c(-c5ccccc5)c...,"{'AP': '[3*]c1ccc(-c2cc([15*])cnc2N)cc1', 'SE'...","{'AP': 'Nc1ncccc1-c1ccccc1', 'SE': 'C[NH+]1CCN...",-11.23,0,2,InChI=1S/C25H26N6O/c1-30-9-11-31(12-10-30)16-1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10185,"<img data-content=""rdkit/molecule"" src=""data:i...",8.878885e+10,8.821697e+12,2.72390,375.474,67.39,"{'AP': 124, 'SE': 38}",CN1CCN(C(=O)c2cc(-c3cnc(N)c(-c4ccccc4)c3)cn2C)CC1,"{'AP': '[3*]c1ccc(-c2cc([15*])cnc2N)cc1', 'SE'...","{'AP': 'Nc1ncccc1-c1ccccc1', 'SE': 'Cn1cccc1C(...",497.25,1,2,InChI=1S/C22H25N5O/c1-25-8-10-27(11-9-25)22(28...
10186,"<img data-content=""rdkit/molecule"" src=""data:i...",1.396197e+11,1.387204e+13,-0.73300,344.393,102.76,"{'AP': 75, 'SE': 278}",C[NH+]1CCN(C(=O)c2cccc(CC(=O)Nc3cnc[nH]3)c2O)CC1,"{'AP': '*n1cnc(NC(=O)C[12*])c1', 'SE': '[21*]O...","{'AP': 'CC(=O)Nc1c[nH]cn1', 'SE': 'C[NH+]1CCN(...",148.50,1,2,InChI=1S/C17H21N5O3/c1-21-5-7-22(8-6-21)17(25)...
10187,"<img data-content=""rdkit/molecule"" src=""data:i...",1.586613e+11,1.576394e+13,2.21430,312.327,89.01,"{'AP': 75, 'SE': 184}",COc1cc2ccn(C(C)=O)c2cc1CC(=O)Nc1cnc[nH]1,"{'AP': '*n1cnc(NC(=O)C[12*])c1', 'SE': '[23*]c...","{'AP': 'CC(=O)Nc1c[nH]cn1', 'SE': 'COc1ccc2c(c...",544.51,1,2,InChI=1S/C16H16N4O3/c1-10(21)20-4-3-11-6-14(23...
10188,"<img data-content=""rdkit/molecule"" src=""data:i...",2.828106e+11,2.809890e+13,3.91480,311.363,59.91,"{'AP': 208, 'SE': 184}",COc1cc2ccn(C(C)=O)c2cc1-c1n[nH]c2ccsc12,"{'AP': '[51*]c1n[nH]c2ccsc12', 'SE': '[23*]c1c...","{'AP': 'c1cc2[nH]ncc2s1', 'SE': 'COc1ccc2c(ccn...",191.75,1,2,InChI=1S/C16H13N3O2S/c1-9(20)19-5-3-10-7-14(21...


In [7]:
# find match
data_ap_zero = data_two_frag[data_two_frag.apply(lambda x: x.fragment_ids['AP'] == 0 and x.fragment_ids['SE'] == 0, axis=1)]

In [8]:
# binding affinity of top 100 compounds
data[data.num_fragments == 2]['BIOSOLVEIT.HYDE_ESTIMATED_AFFINITY_LOWER_BOUNDARY [nM]'].sort_values().reset_index(drop=True)[300]

5.291217